In [8]:
import string
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document
from langchain.chains import create_retrieval_chain

llm = Ollama(model='llama2')

loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
embeddings = OllamaEmbeddings()
docs = loader.load()

output_parser = StrOutputParser()
text_splitter = RecursiveCharacterTextSplitter()

documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

prompt = ChatPromptTemplate.from_template({
    ("input", "{input}"),
    ("context", [Document(page_content="You are a world class programmer and you have been tasked with helping build a new feature for a software application.")]),
})

chain = prompt | llm | output_parser

response: string = chain.invoke({"input": "how can langsmith help with building the new feature?"})


"\nAs a world-class programmer, I can offer several ways in which I can help build the new feature for your software application:\n\n1. Design and Architecture: I can assist in designing the architecture of the feature, including the overall structure, modules, and APIs. This will ensure that the feature is scalable, maintainable, and easy to integrate with the existing system.\n2. Technical Leadership: As a technical lead, I can guide the development team in building the feature. I can provide technical direction, make strategic decisions, and ensure that the project stays on track and within budget.\n3. Code Development: I can write high-quality code for the new feature, using modern programming languages and technologies. I can also review and optimize the code of other developers to ensure that it meets the highest standards of quality and performance.\n4. Testing and Quality Assurance: I can help write automated tests to ensure that the new feature works as expected. I can also pe